In [1]:
1+1

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2

In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from mosaicml import *
from mosaicml.constants import MLModelFlavours
from werkzeug.datastructures import FileStorage
import pandas as pd
import os, tempfile

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# import time
# time.sleep(1000)
# print("-----creating spark session")
# spark = SparkSession.builder.appName("jupyter-nb-test").getOrCreate()
# spark.sparkContext.setLogLevel('WARN')

print("-----reading data from azure")
storage_account_name = "pocmosaic"
storage_account_access_key = "iX6jZejeNh1bUejdSRYk+4X2S/pAbTy+idkTM5UAw2HxgxSnaYGJSN+ffzZAQ5/Q6xQe2Ja74Re9nKHMVmsGhA=="
file_location = "wasbs://container1@pocmosaic.blob.core.windows.net/catalog/local_file_upload/RatanBBostonHousing.csv"
file_type = "csv"
spark.conf.set("fs.azure.account.key."+storage_account_name+".blob.core.windows.net",storage_account_access_key)
dataset = spark.read.option("header", "true").format(file_type).option("inferSchema", "true").load(file_location)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----reading data from azure

In [4]:
dataset.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

In [5]:
print("-----splitting data")
train_data,test_data = dataset.randomSplit([0.8,0.2])

print("-----creating pipeline")
stage_1 = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')
# define stage 5: logistic regression model                          
stage_2 = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

# setup the pipeline
regression_pipeline = Pipeline(stages= [stage_1, stage_2])
print("-----training pipeline")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----splitting data
-----creating pipeline
-----training pipeline

In [6]:
regression_pipeline = regression_pipeline.fit(train_data)
test_update = regression_pipeline.transform(test_data)
test_update.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|   nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|          Attributes|        prediction|
+-------+----+-----+----+------+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|0.00906|90.0| 2.97|   0|   0.4|7.088|20.8|7.3073|  1|285|   15.3|394.72| 7.85|32.2|[0.00906,90.0,2.9...|31.769392402482644|
|0.01311|90.0| 1.22|   0| 0.403|7.249|21.9|8.6966|  5|226|   17.9|395.93| 4.81|35.4|[0.01311,90.0,1.2...|31.226946218281377|
|0.01501|80.0| 2.01|   0| 0.435|6.635|29.7| 8.344|  4|280|   17.0|390.94| 5.99|24.5|[0.01501,80.0,2.0...|27.795412335709397|
|0.01501|90.0| 1.21|   1| 0.401|7.923|24.8| 5.885|  1|198|   13.6|395.52| 3.16|50.0|[0.01501,90.0,1.2...| 43.52282299407406|
|0.01951|17.5| 1.38|   0|0.4161|7.104|59.5|9.2229|  3|216|   18.6|393.24| 8.05|33.0|[0.01951,17.5,1.3...|23.308006211986374|


In [ ]:
#Predict using the model
print("-----test data")
test_update.show()

# define score function
def score(model, request):
    data = request.json
    df = pd.DataFrame(data, columns=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat', 'medv'])
    spark = SparkSession.builder.appName("jupyter-nb-test").getOrCreate()
    spark_df = spark.createDataFrame(df)
    return model.transform(spark_df).toPandas()["prediction"].to_json()

print("-----defining & testing score function")
import requests
req = requests.Request()
req.json = [[1, -0.1, 0.0425, 0.0279, 2.4716, -15.2808, 4.2783, -0.0005, -1.4009, 0.2692, -0.013, -0.9905, 0.0084, -0.5533]]
rez = score(regression_pipeline, req)
print("the prediction is {}".format(rez))

print("-----register model")
register_model(regression_pipeline, score, "Boston_Regression_Analysis_jun30", "pyspark model", MLModelFlavours.pyspark)